In [1]:
import pandas as pd
import h5py
import numpy
import re
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split

In [2]:
TRAIN_FILE = "train.h5"
TEST_FILE = "test.h5"
BASELINE_WIENER = "baseline_wiener.csv"
BASELINE_KALMAN = "baseline_kalman.csv"
EMG_FILTER_LAG = 2

In [3]:
def compute_differentials(array, order, axis):
    results = [array]
    for _ in range(order):
        results.append(numpy.diff(results[-1], axis=axis))
    return results

In [4]:
def align_arrays(arrays_list):
    final_size = min(map(len, arrays_list))
    return [array[-final_size:] for array in arrays_list]

In [5]:
def cut_arrays(arrays_list, head=None, tail=None):
    return [array[head:tail] for array in arrays_list]

In [6]:
def transform_emg(dataset):
    """For a dataset[n_ticks, n_emg_channels] computes a transforamtion with
    added time-shifted channels dataset[n_ticks -  2*EMG_FILTER_LAG, 3*n_emg_channels] by
    by pasting the emg values for the past EMG_FILTER_LAG ticks as additional channels."""
    emg_channels = dataset.shape[1]
    emg_lag = numpy.zeros((dataset.shape[0] - 2*EMG_FILTER_LAG, emg_channels*3))
    for l in range(EMG_FILTER_LAG+1):
        emg_lag[:, l*emg_channels:(l+1)*emg_channels] = dataset[EMG_FILTER_LAG-1+l:-EMG_FILTER_LAG-1+l]
    return emg_lag

In [7]:
class BaselineModel(object):
    def fit(self, emg_list, coordinates_and_diffs_list):
        transformed_emg_list = list(map(transform_emg, emg_list))
        emg_lag = coordinates_and_diffs_list[0].shape[0] - transformed_emg_list[0].shape[0]
        assert(emg_lag == EMG_FILTER_LAG)
        coordinates_and_diffs_uncut = numpy.vstack(cut_arrays(coordinates_and_diffs_list, head=emg_lag))
        coordinates_and_diffs_lag = numpy.vstack(cut_arrays(
                coordinates_and_diffs_list, head=emg_lag, tail=-1))
        coordinates_and_diffs_now = numpy.vstack(cut_arrays(
                coordinates_and_diffs_list, head=1 + emg_lag))
        emg = numpy.vstack(transformed_emg_list)
        emg = numpy.hstack((numpy.ones((emg.shape[0], 1)), emg))
        self.W = numpy.linalg.pinv(emg).dot(coordinates_and_diffs_uncut).T
        measurment_error = coordinates_and_diffs_uncut - emg.dot(self.W.T)
        # To be used in the Kalman Filter as Ez, measurement noise
        self.measurment_error_covar = numpy.cov(measurment_error.T)
        
        self.A = numpy.linalg.pinv(coordinates_and_diffs_lag).dot(coordinates_and_diffs_now).T
        state_trans_error = coordinates_and_diffs_now - coordinates_and_diffs_lag.dot(self.A.T)
        # To be used in the Kalman Filter as Ex, process noise
        self.state_trans_covar = numpy.cov(state_trans_error.T)
    
    def predict(self, emg):
        transformed_emg = transform_emg(emg)
        transformed_emg = numpy.hstack((numpy.ones((transformed_emg.shape[0], 1)), transformed_emg))
        ticks_count = transformed_emg.shape[0]
        # TODO(kazeevn) how magic is 6?
        # We add zero preictions for the points we can't predict due to the lag
        # An exercise for the reader a better estimate is
        kalman_estimate = numpy.zeros((ticks_count + 2*EMG_FILTER_LAG, 6))
        wiener_estimate = numpy.zeros((ticks_count + 2*EMG_FILTER_LAG, 6))
        kalman_tick = numpy.zeros((6, 1))
        p_after = self.state_trans_covar.copy()
        for tick in range(1, transformed_emg.shape[0]):
            # Predict coordinate by state transition equation
            # TODO(kazeevn) why the name?
            x_state_estimate = self.A.dot(kalman_tick)
            # Predict MSE covarimance matrix estimate
            p_before = self.A.dot(p_after.dot(self.A.T)) + self.state_trans_covar
            # Predict coordinate by state measurement equation
            x_measurment_estimate = self.W.dot(transformed_emg[tick].T)[numpy.newaxis, :]
            p_after = numpy.linalg.pinv(numpy.linalg.pinv(p_before) +
                                        numpy.linalg.pinv(self.measurment_error_covar))
            # Update the state estimate
            kalman_tick = p_after.dot(numpy.linalg.pinv(p_before).dot(x_state_estimate) + 
                                          numpy.linalg.pinv(self.measurment_error_covar).dot(
                                            x_measurment_estimate.T))
            
            # Store the predicted coordinates
            kalman_estimate[tick + 2*EMG_FILTER_LAG] = kalman_tick.T
            wiener_estimate[tick + 2*EMG_FILTER_LAG] = x_measurment_estimate
        return (wiener_estimate, kalman_estimate)

## Validation

In [8]:
def score_trial(true_coordinates, predicted_coordinates, dimensions=2):
    return numpy.mean([pearsonr(numpy.diff(true_coordinates[:, axis]), 
                                numpy.diff(predicted_coordinates[:, axis]))[0] for
                       axis in range(dimensions)])

In [9]:
numbergetter = re.compile(r"\d+$")
def get_tail_number(string):
    return int(numbergetter.findall(string)[0])

In [10]:
shifts = [0, 1]
subject_ids = []
trials = []
wiener = []
kalman = []
with h5py.File(TRAIN_FILE, "r") as train_io, h5py.File(TEST_FILE, "r") as test_io:
    for subject_id, subject_data in train_io.items():
        coordinates_and_diffs_list_all = []
        emg_list = []
        for digit, subject_trials in subject_data.items():
            for trial in subject_trials.values():
                pen_coordintes_with_emg_lag = trial['pen_coordinates']
                coordinates_and_diffs_list_all.append(numpy.hstack(align_arrays(
                        compute_differentials(pen_coordintes_with_emg_lag, 2, 0))))
                emg_list.append(numpy.array(trial['emg']))
                assert(trial['emg'].shape[0] == trial['pen_coordinates'].shape[0])
        # Validataion
        model = BaselineModel()
        emg_train, emg_test, coordinates_and_diffs_list_train, coordinates_and_diffs_list_test = \
            train_test_split(emg_list, coordinates_and_diffs_list_all)
        model.fit(emg_train, coordinates_and_diffs_list_train)
        score_wiener = 0
        score_kalman = 0
        for trial_emg, trial_coordinates_and_diffs in zip(emg_test, coordinates_and_diffs_list_test):
            wiener_estimate, kalman_estimate = model.predict(trial_emg)
            # 2: is due to aligning of the arrays while computing the differentials.
            # For a perfectly coorrect validation (like in the contest) it of course should
            # be done over the entire arrays
            score_wiener += score_trial(trial_coordinates_and_diffs, wiener_estimate[2:])
            score_kalman += score_trial(trial_coordinates_and_diffs, kalman_estimate[2:])
        validation_count = len(emg_test)
        print("Mean scores; Kalman: %f, Wiener: %f" % (score_kalman/validation_count, score_wiener/validation_count))
        # Prediction
        model = BaselineModel()
        model.fit(emg_list, coordinates_and_diffs_list_all)
        for trial_name, trial_data in test_io[subject_id].items():
            trials.append(get_tail_number(trial_name))
            subject_ids.append(get_tail_number(subject_id))
            wiener_estimate, kalman_estimate = model.predict(trial_data)
            kalman.append(kalman_estimate)
            wiener.append(wiener_estimate)

Mean scores; Kalman: 0.571216, Wiener: 0.446204
Mean scores; Kalman: 0.448462, Wiener: 0.239530
Mean scores; Kalman: 0.523814, Wiener: 0.368038


In [11]:
def write_solution(trial_indexes, trajectories, file_name):
    solution_list = numpy.vstack([
            [[subject_id, trial_index, tick, xy[0], xy[1]] for tick, xy in enumerate(trajectory)] for
            subject_id, trial_index, trajectory in zip(subject_ids, trial_indexes, trajectories)
        ])
    solution = pd.DataFrame.from_records(solution_list, columns=["subject_id", "trial_id", "tick_index", "x", "y"])
    solution.to_csv(file_name, index=False)

In [12]:
write_solution(trials, kalman, BASELINE_KALMAN)
write_solution(trials, wiener, BASELINE_WIENER)